In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
prr_df = pd.read_csv("nfl_pass_rush_receive_raw_data.csv")

In [7]:
positions = prr_df['pos'].unique().copy()

columns = ['pass_cmp', 'pass_att', 'pass_yds',
       'pass_td', 'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_long',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_td', 'rush_long',
       'targets', 'rec', 'rec_yds', 'rec_td', 'rec_long', 'fumbles_lost', 'rush_yac', 'rec_yac', 'rec_drops']
agg = {}
for column in columns:
    if column in ['pass_rating', 'off_pct']:
        agg[column] = 'mean'
    elif column in ['pass_long', 'rush_long']:
        agg[column] = 'max'
    else:
        agg[column] = 'sum'
        
pos_df = pd.DataFrame()

for pos in positions:
    result = prr_df[prr_df['pos']==pos]\
        .groupby('player', as_index=False)\
        .agg(agg)
    result.insert(1, column='position', value=pos)
    pos_df = pd.concat([result, pos_df], ignore_index=True)

player_id = []
teams = []
for pid in pos_df['player']:
    player_id.append(prr_df[prr_df['player']==pid]['player_id'].iloc[0])
    teams.append(prr_df[prr_df['player']==pid]['team'].iloc[0])


pos_df.insert(0, column='player_id', value=player_id)
pos_df.insert(3, column='team', value=teams)

pos_df.head()

,player_id,player,position,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,...,rush_long,targets,rec,rec_yds,rec_td,rec_long,fumbles_lost,rush_yac,rec_yac,rec_drops
0,BradGa00,Garrett Bradbury,C,MIN,0,0,0,0,0,0,...,0,1,1,21,0,21,0,0,24,0
1,WilkCh00,Christian Wilkins,DT,MIA,0,0,0,0,0,0,...,0,1,1,1,1,1,0,0,0,0
2,BellNi01,Nick Bellore,LB,SEA,0,0,0,0,0,0,...,0,1,1,4,0,4,0,0,4,0
3,FejeCl00,Clayton Fejedelem,SS,MIA,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,ChanSe00,Sean Chandler,SS,CAR,0,0,0,0,0,0,...,14,0,0,0,0,0,0,0,0,0


In [8]:
pos_df.to_csv('nfl_offense_data_2021.csv', index=False)